<a href="https://colab.research.google.com/github/chemicoPy/COVID-19_Data_visualization/blob/main/COVID19_Data_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

In [ ]:
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!pip install plotly
!pip install requests

In [ ]:
import numpy as np
from numpy import *
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import pandas as pd
from pandas import DataFrame, Series
import math
import io
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import requests
from datetime import datetime

In [ ]:
raw = requests.get("https://services1.arcgis.com/0MSEUqKaxRlEPj5g/arcgis/rest/services/Coronavirus_2019_nCoV_Cases/FeatureServer/1/query?where=1%3D1&outFields=*&outSR=4326&f=json")
raw_json = raw.json()
data = pd.DataFrame(raw_json["features"])

data_list = data["attributes"].tolist()
dataset = pd.DataFrame(data_list)
dataset.set_index("OBJECTID")
dataset = dataset[["Country_Region", "Province_State", "Lat", "Long_", "Confirmed", "Deaths", "Recovered", "Last_Update"]]


def convertTime(t):
    t = int(t)
    return datetime.fromtimestamp(t)

dataset = dataset.dropna(subset=["Last_Update"])
dataset["Province_State"].fillna(value="", inplace=True)

dataset["Last_Update"]= dataset["Last_Update"]/1000
dataset["Last_Update"] = dataset["Last_Update"].apply(convertTime)

In [ ]:
dataset

In [ ]:
dataset.describe()

,Lat,Long_,Confirmed,Deaths,Recovered
count,766.000000,766.000000,7.850000e+02,785.000000,785.0
mean,26.988002,12.217503,3.086482e+05,6274.973248,0.0
std,25.400907,77.999261,7.813595e+05,15622.063892,0.0
min,-52.368000,-178.116500,0.000000e+00,0.000000,0.0
25%,9.315100,-67.826400,1.680500e+04,207.000000,0.0
50%,33.869446,14.812800,7.295600e+04,1544.000000,0.0
75%,47.181489,73.437670,2.396300e+05,5878.000000,0.0
max,71.706900,178.065000,7.744109e+06,151297.000000,0.0


In [ ]:
df_total = dataset.groupby("Country_Region", as_index=False).agg(
    {
        "Confirmed" : "sum",
        "Deaths" : "sum",
        "Recovered" : "sum"
    }
)

In [ ]:
df_total

In [ ]:
total_confirmed = dataset["Confirmed"].sum()
total_recovered = dataset["Recovered"].sum()
total_deaths = dataset["Deaths"].sum()

In [ ]:
df_top10 = df_total.nlargest(10, "Confirmed")
top10_countries_1 = df_top10["Country_Region"].tolist()
top10_confirmed = df_top10["Confirmed"].tolist()

df_top10 = df_total.nlargest(10, "Recovered")
top10_countries_2 = df_top10["Country_Region"].tolist()
top10_recovered = df_top10["Recovered"].tolist()

df_top10 = df_total.nlargest(10, "Deaths")
top10_countries_3 = df_top10["Country_Region"].tolist()
top10_deaths = df_top10["Deaths"].tolist()

In [ ]:
df_top10

In [ ]:

fig1 = make_subplots( rows = 4, cols = 6,
    specs = [
            [{"type": "scattergeo", "rowspan": 4, "colspan": 3}, None, None, {"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"} ],
            [    None, None, None,               {"type": "bar", "colspan":3}, None, None],
            [    None, None, None,              {"type": "bar", "colspan":3}, None, None],
            [    None, None, None,               {"type": "bar", "colspan":3}, None, None],
          ]
)



In [ ]:
message = dataset["Country_Region"] + " " + dataset["Province_State"] + "<br>"
message += "Confirmed: " + dataset["Confirmed"].astype(str) + "<br>"
message += "Deaths: " + dataset["Deaths"].astype(str) + "<br>"
message += "Recovered: " + dataset["Recovered"].astype(str) + "<br>"
message += "Last updated: " + dataset["Last_Update"].astype(str)
dataset["text"] = message

In [ ]:
fig1.add_trace(
    go.Scattergeo(
        locationmode = "country names",
        lon = dataset["Long_"],
        lat = dataset["Lat"],
        hovertext = dataset["text"],
        showlegend=False,
        marker = dict(
            size = 10,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = True,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            cmin = 0,
            color = dataset['Confirmed'],
            cmax = dataset['Confirmed'].max(),
            colorbar_title="Confirmed Cases<br>Latest Update",  
            colorbar_x = -0.05
        )

    ),
    
    row=1, col=1
)



In [ ]:
""" And for Top 10 """


fig2 = make_subplots( rows = 4, cols = 6,
    specs = [
            [{"type": "scattergeo", "rowspan": 4, "colspan": 3}, None, None, {"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"} ],
            [    None, None, None,               {"type": "bar", "colspan":3}, None, None],
            [    None, None, None,              {"type": "bar", "colspan":3}, None, None],
            [    None, None, None,               {"type": "bar", "colspan":3}, None, None],
          ]
)


message += "Confirmed: " + df_top10["Confirmed"].astype(str) + "<br>"
message += "Deaths: " + df_top10["Deaths"].astype(str) + "<br>"
message += "Recovered: " + df_top10["Recovered"].astype(str) + "<br>"
df_top10["text"] = message



In [ ]:
fig2.add_trace(
    go.Scattergeo(
        locationmode = "country names",
        hovertext = df_top10["text"],
        showlegend=False,
        marker = dict(
            size = 10,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = True,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            cmin = 0,
            color = df_top10['Confirmed'],
            cmax = df_top10['Confirmed'].max(),
            colorbar_title="Confirmed Cases<br>Latest Update",  
            colorbar_x = -0.05
        )

    ),
    
    row=1, col=1
)

fig2.update_layout(
    xaxis=dict(
        rangeslider=dict (
            visible=True
        )
    )
)


In [ ]:
""" View of summary plots ""

In [ ]:
fig1.add_trace(
    go.Indicator(
        mode="number",
        value=total_confirmed,
        title="Confirmed Cases",
    ),
    row=1, col=4
)

fig1.add_trace(
    go.Indicator(
        mode="number",
        value=total_recovered,
        title="Recovered Cases",
    ),
    row=1, col=5
)

fig1.add_trace(
    go.Indicator(
        mode="number",
        value=total_deaths,
        title="Deaths Cases",
    ),
    row=1, col=6
)

In [ ]:
fig1.add_trace(
    go.Bar(
        x=top10_countries_1,
        y=top10_confirmed, 
        name= "Confirmed Cases",
        marker=dict(color="Yellow"), 
        showlegend=True,
    ),
    row=2, col=4
)

fig1.add_trace(
    go.Bar(
        x=top10_countries_2,
        y=top10_recovered, 
        name= "Recovered Cases",
        marker=dict(color="Green"), 
        showlegend=True),
    row=3, col=4
)

fig1.add_trace(
    go.Bar(
        x=top10_countries_3,
        y=top10_deaths, 
        name= "Deaths Cases",
        marker=dict(color="crimson"), 
        showlegend=True),
    row=4, col=4
)

In [ ]:
fig1.update_layout(
    template="plotly_dark",
    title = "Global COVID-19 Cases (Last Updated: " + str(dataset["Last_Update"][0]) + ")",
    showlegend=True,
    legend_orientation="h",
    legend=dict(x=0.65, y=0.8),
    geo = dict(
            projection_type="orthographic",
            showcoastlines=True,
            landcolor="white", 
            showland= True,
            showocean = True,
            lakecolor="LightBlue"
    ),

    annotations=[
        dict(
            showarrow=False,
            xref="paper",
            yref="paper",
            x=0.35,
            y=0)
    ]
)